In [24]:
import pandas as pd
import numpy as np
import platform
from IPython.display import display, HTML

from sklearn.model_selection import StratifiedKFold, cross_val_score

import pickle

In [2]:
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)

In [3]:
if platform.os == 'Windows':
    path = 'Data\\'
else:
    path = 'Data//'

In [4]:
df = pd.read_csv(path+'dados_tratados.csv', encoding='latin1')

In [5]:
df.head()

,SIGLA_UF,CODIGO_CARGO,DESCRICAO_CARGO,NOME_CANDIDATO,SEQUENCIAL_CANDIDATO,COD_SITUACAO_CANDIDATURA,DES_SITUACAO_CANDIDATURA,NUMERO_PARTIDO,CODIGO_LEGENDA,CODIGO_OCUPACAO,DESCRICAO_OCUPACAO,IDADE_DATA_ELEICAO,CODIGO_SEXO,DESCRICAO_SEXO,COD_GRAU_INSTRUCAO,DESCRICAO_GRAU_INSTRUCAO,CODIGO_ESTADO_CIVIL,DESCRICAO_ESTADO_CIVIL,CODIGO_COR_RACA,DESCRICAO_COR_RACA,CODIGO_NACIONALIDADE,DESCRICAO_NACIONALIDADE,DESPESA_MAX_CAMPANHA,COD_SIT_TOT_TURNO,DESC_SIT_TOT_TURNO,VALOR_BEM,setor_A,setor_B,setor_C,setor_D,setor_E,setor_F,setor_G,setor_H,setor_I,setor_J,setor_K,setor_L,setor_M,setor_N,setor_O,setor_P,setor_Q,setor_R,setor_S,setor_nao_identificado,tp_receita_aplicacao,tp_receita_evento,tp_receita_fisica,tp_receita_internet,tp_receita_juridica,tp_receita_nao_identificada,tp_receita_outro,tp_receita_partido,tp_receita_proprio,valor_receita
0,AC,7,DEPUTADO ESTADUAL,BENEDITO SILVA BARBOSA,10000000504,2,DEFERIDO,23,10000000034,266,PROFESSOR DE ENSINO MÉDIO,46,2,MASCULINO,8,SUPERIOR COMPLETO,9,DIVORCIADO(A),3,PARDA,1,BRASILEIRA NATA,1000000,5,SUPLENTE,210000.00,0.00,0.00,0.00,0.00,0.00,490.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,200.00,1500.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,690.50,1500.00,2190.50
1,AC,7,DEPUTADO ESTADUAL,MARIA APARECIDA ALVES DE SOUZA,10000000055,2,DEFERIDO,12,10000000021,254,VIGILANTE,32,4,FEMININO,6,ENSINO MÉDIO COMPLETO,1,SOLTEIRO(A),3,PARDA,1,BRASILEIRA NATA,600000,5,SUPLENTE,38918.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,821.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,821.88,0.00,0.00,821.88
2,AC,7,DEPUTADO ESTADUAL,ASTÉRIO DE PAULA MOREIRA FILHO,10000000206,2,DEFERIDO,51,10000000028,277,DEPUTADO,53,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),3,PARDA,1,BRASILEIRA NATA,600000,5,SUPLENTE,493000.00,0.00,0.00,10000.00,0.00,0.00,0.00,20400.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23605.00,52600.00,0.00,0.00,18100.00,0.00,0.00,0.00,36505.00,17500.00,34500.00,106605.00
3,AC,7,DEPUTADO ESTADUAL,JOSÉ MASTRANGELO FILHO,10000000514,2,DEFERIDO,23,10000000034,171,JORNALISTA E REDATOR,30,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),1,BRANCA,1,BRASILEIRA NATA,1000000,5,SUPLENTE,20000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6500.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6500.00,6500.00
4,AC,7,DEPUTADO ESTADUAL,CARLOS ANDRÉ MENEZES DE OLIVEIRA,10000000519,2,DEFERIDO,23,10000000034,244,TÉCNICO EM EDIFICAÇÕES,34,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,1,SOLTEIRO(A),1,BRANCA,1,BRASILEIRA NATA,1000000,5,SUPLENTE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [6]:
df[['COD_SIT_TOT_TURNO', 'DESC_SIT_TOT_TURNO']].groupby(['COD_SIT_TOT_TURNO', 'DESC_SIT_TOT_TURNO']).count()


,
COD_SIT_TOT_TURNO,DESC_SIT_TOT_TURNO
2,ELEITO POR QP
3,ELEITO POR MÉDIA
4,NÃO ELEITO
5,SUPLENTE


In [7]:
print ('Deputados Federais')
df_federais = df[df['CODIGO_CARGO'] == 6]
quantidade_candidatos = df_federais[df_federais['COD_SITUACAO_CANDIDATURA'] == 2].shape[0]
quantidade_eleitos = df_federais[(df_federais['COD_SIT_TOT_TURNO'] == 2) | (df_federais['COD_SIT_TOT_TURNO'] == 3)].shape[0]

print ('Candidatos:', quantidade_candidatos)
print ('Eleitos:', quantidade_eleitos, ((quantidade_eleitos / quantidade_candidatos)*100),'%')

#http://www2.camara.leg.br/deputados/pesquisa/bancadas/bancada-na-eleicao

Deputados Federais
Candidatos: 5862
Eleitos: 513 8.751279426816785 %


In [8]:
print ('Deputados Estaduais / Distritais')
df_estaduais = df[df['CODIGO_CARGO'] != 6]
quantidade_candidatos = df_estaduais[df_estaduais['COD_SITUACAO_CANDIDATURA'] == 2].shape[0]
quantidade_eleitos = df_estaduais[(df_estaduais['COD_SIT_TOT_TURNO'] == 2) | (df_estaduais['COD_SIT_TOT_TURNO'] == 3)].shape[0]

print ('Candidatos:', quantidade_candidatos)
print ('Eleitos:', quantidade_eleitos, ((quantidade_eleitos / quantidade_candidatos)*100),'%')


Deputados Estaduais / Distritais
Candidatos: 15239
Eleitos: 1059 6.949274886803596 %


In [9]:
print ('Total deputados')
quantidade_candidatos = df[df['COD_SITUACAO_CANDIDATURA'] == 2].shape[0]
quantidade_eleitos = df[(df['COD_SIT_TOT_TURNO'] == 2) | (df['COD_SIT_TOT_TURNO'] == 3)].shape[0]

print ('Candidatos:', quantidade_candidatos)
print ('Eleitos:', quantidade_eleitos, ((quantidade_eleitos / quantidade_candidatos)*100),'%')


Total deputados
Candidatos: 21101
Eleitos: 1572 7.449883891758684 %


In [10]:
df.drop('DESCRICAO_CARGO', axis=1, inplace=True)
df.drop('NOME_CANDIDATO', axis=1, inplace=True)
df.drop('SEQUENCIAL_CANDIDATO', axis=1, inplace=True)
df.drop('COD_SITUACAO_CANDIDATURA', axis=1, inplace=True)
df.drop('DES_SITUACAO_CANDIDATURA', axis=1, inplace=True)
df.drop('NUMERO_PARTIDO', axis=1, inplace=True)
df.drop('CODIGO_LEGENDA', axis=1, inplace=True)
df.drop('DESCRICAO_OCUPACAO', axis=1, inplace=True)
df.drop('DESCRICAO_SEXO', axis=1, inplace=True)
df.drop('DESCRICAO_GRAU_INSTRUCAO', axis=1, inplace=True)
df.drop('DESCRICAO_ESTADO_CIVIL', axis=1, inplace=True)
df.drop('DESCRICAO_COR_RACA', axis=1, inplace=True)
df.drop('CODIGO_NACIONALIDADE', axis=1, inplace=True)
df.drop('DESCRICAO_NACIONALIDADE', axis=1, inplace=True)
df.drop('COD_SIT_TOT_TURNO', axis=1, inplace=True)
df.drop('SIGLA_UF', axis=1, inplace=True)

In [11]:
df.head()

,CODIGO_CARGO,CODIGO_OCUPACAO,IDADE_DATA_ELEICAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,CODIGO_ESTADO_CIVIL,CODIGO_COR_RACA,DESPESA_MAX_CAMPANHA,DESC_SIT_TOT_TURNO,VALOR_BEM,setor_A,setor_B,setor_C,setor_D,setor_E,setor_F,setor_G,setor_H,setor_I,setor_J,setor_K,setor_L,setor_M,setor_N,setor_O,setor_P,setor_Q,setor_R,setor_S,setor_nao_identificado,tp_receita_aplicacao,tp_receita_evento,tp_receita_fisica,tp_receita_internet,tp_receita_juridica,tp_receita_nao_identificada,tp_receita_outro,tp_receita_partido,tp_receita_proprio,valor_receita
0,7,266,46,2,8,9,3,1000000,SUPLENTE,210000.00,0.00,0.00,0.00,0.00,0.00,490.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,200.00,1500.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,690.50,1500.00,2190.50
1,7,254,32,4,6,1,3,600000,SUPLENTE,38918.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,821.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,821.88,0.00,0.00,821.88
2,7,277,53,2,6,3,3,600000,SUPLENTE,493000.00,0.00,0.00,10000.00,0.00,0.00,0.00,20400.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23605.00,52600.00,0.00,0.00,18100.00,0.00,0.00,0.00,36505.00,17500.00,34500.00,106605.00
3,7,171,30,2,8,3,1,1000000,SUPLENTE,20000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6500.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6500.00,6500.00
4,7,244,34,2,6,1,1,1000000,SUPLENTE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [12]:
df.shape

(21124, 40)

In [13]:
df['DESC_SIT_TOT_TURNO'].unique()

array(['SUPLENTE', 'ELEITO POR QP', 'NÃO ELEITO', 'ELEITO POR MÉDIA'], dtype=object)

In [14]:
s = {'SUPLENTE':0, 'ELEITO POR QP':1, 'NÃO ELEITO':0, 'ELEITO POR MÉDIA':1}
df['DESC_SIT_TOT_TURNO'] = df['DESC_SIT_TOT_TURNO'].map(s)
display(df)

,CODIGO_CARGO,CODIGO_OCUPACAO,IDADE_DATA_ELEICAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,CODIGO_ESTADO_CIVIL,CODIGO_COR_RACA,DESPESA_MAX_CAMPANHA,DESC_SIT_TOT_TURNO,VALOR_BEM,setor_A,setor_B,setor_C,setor_D,setor_E,setor_F,setor_G,setor_H,setor_I,setor_J,setor_K,setor_L,setor_M,setor_N,setor_O,setor_P,setor_Q,setor_R,setor_S,setor_nao_identificado,tp_receita_aplicacao,tp_receita_evento,tp_receita_fisica,tp_receita_internet,tp_receita_juridica,tp_receita_nao_identificada,tp_receita_outro,tp_receita_partido,tp_receita_proprio,valor_receita
0,7,266,46,2,8,9,3,1000000,0,210000.00,0.00,0.00,0.00,0.00,0.00,490.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,200.00,1500.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,690.50,1500.00,2190.50
1,7,254,32,4,6,1,3,600000,0,38918.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,821.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,821.88,0.00,0.00,821.88
2,7,277,53,2,6,3,3,600000,0,493000.00,0.00,0.00,10000.00,0.00,0.00,0.00,20400.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23605.00,52600.00,0.00,0.00,18100.00,0.00,0.00,0.00,36505.00,17500.00,34500.00,106605.00
3,7,171,30,2,8,3,1,1000000,0,20000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6500.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6500.00,6500.00
4,7,244,34,2,6,1,1,1000000,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,7,131,61,2,8,3,3,1000000,0,540000.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,7,999,29,4,8,1,3,900000,0,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1640.00,6000.00,0.00,0.00,6000.00,0.00,0.00,0.00,1640.00,0.00,0.00,7640.00
7,7,297,46,2,8,3,3,500000,0,105000.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,7,171,41,2,7,3,2,600000,0,90000.00,0.00,0.00,0.00,0.00,0.00,0.00,1500.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3660.00,3000.00,0.00,0.00,0.00,0.00,1500.00,0.00,3660.00,0.00,3000.00,8160.00
9,7,999,47,4,3,5,3,600000,0,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,911.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,911.00,0.00,0.00,911.00


In [15]:
print (df['VALOR_BEM'].sum())

9102898393.660002


In [16]:
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import *

In [17]:
def metrics(y_test, y_pred):
    print('acc:', accuracy_score(y_test, y_pred))
    print('precision: ', precision_score(y_test, y_pred))
    print('recall: ', recall_score(y_test, y_pred))
    print('f1:', f1_score(y_test, y_pred))
    print('ROC:', roc_auc_score(y_test, y_pred))


In [18]:
df.fillna(0,inplace=True)

In [19]:
def remove_columns(df, columns_to_remove):
    return df[[x for x in list(df.columns) if x not in columns_to_remove]]
        

In [20]:
df = remove_columns(df, ['setor_A', 'setor_B', 'setor_C', 'setor_D', 'setor_E',
       'setor_F', 'setor_G', 'setor_H', 'setor_I', 'setor_J', 'setor_K',
       'setor_L', 'setor_M', 'setor_N', 'setor_O', 'setor_P', 'setor_Q',
       'setor_R', 'setor_S', 'setor_nao_identificado', 'tp_receita_aplicacao',
       'tp_receita_evento', 'tp_receita_fisica', 'tp_receita_internet',
       'tp_receita_juridica', 'tp_receita_nao_identificada',
       'tp_receita_outro', 'tp_receita_partido', 'tp_receita_proprio'])

In [21]:

y = df['DESC_SIT_TOT_TURNO']
df.drop('DESC_SIT_TOT_TURNO', axis=1, inplace=True)
X = df


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [25]:
clf = GaussianNB()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
metrics(y_test, y_pred)

acc: 0.941109638326
precision:  0.646643109541
recall:  0.464467005076
f1: 0.540620384047
ROC: 0.722002276837
